# Data

## Libraries for Data Processing

In [ ]:
%matplotlib inline

String processing and utility libraries

In [ ]:
import os
import glob
import string
import re
import imageio

Libraries for functional programming

In [ ]:
import operator as op
import itertools as it
from functools import reduce, partial
import toolz as tz
import toolz.curried as c

Libraries for numerical programming

In [ ]:
import numpy as np
import pandas as pd
from scipy import (stats, sparse, linalg, 
                   spatial, integrate, optimize, io)

Libraries for plotting and visualization

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

## Working with unstructured text data

### Categorical and one-hot encoding

See examples from S05_Text notebook.

### Term frequency - inverse document frequency (tf-idf)

> In information retrieval, tf–idf or TFIDF, short for term frequency–inverse document frequency, is a numerical statistic that is intended to reflect how important a word is to a document in a collection or corpus. It is often used as a weighting factor in searches of information retrieval, text mining, and user modeling. The tf-idf value increases proportionally to the number of times a word appears in the document, but is often offset by the frequency of the word in the corpus, which helps to adjust for the fact that some words appear more frequently in general. Nowadays, tf-idf is one of the most popular term-weighting schemes; 83% of text-based recommender systems in the domain of digital libraries use tf-idf.

Source: [Wikipedia](https://en.wikipedia.org/wiki/Tf–idf)

#### Documents 

In [ ]:
paths = glob.glob(os.path.join('data', 'Gutenberg', '*.txt') )
names = [os.path.splitext(os.path.split(path)[-1])[0] for path in paths]
names

In [ ]:
N = len(names)

#### Simple processing to find words in each document

Standard Python idiom.

In [ ]:
doc_terms = []
for path in paths:
    with open(path, encoding='latin-1') as f:
        text = f.read()
        text = text.lower()
        text = text.translate(str.maketrans('', '', string.punctuation))
        words = text.split()
        doc_terms.append(words)

Using a functional idiom

In [ ]:
doc_terms = tz.pipe(
    paths,
    c.map(partial(open, encoding='latin-1')),
    c.map(lambda x: x.read()),
    c.map(lambda x: x.lower()),
    c.map(lambda x: x.translate(str.maketrans('', '', string.punctuation))),
    c.map(lambda x: x.split()),
    list
)

#### Term frequency

This is just a word count for each document.

In [ ]:
tf = {name: tz.frequencies(doc) 
      for name, doc in zip(names, doc_terms)}

#### Document frequency

This is how many documents a term appears in.

In [ ]:
df = tz.frequencies(tz.concat(d.keys() for d in tf.values()))

#### Inverse document frequency 

This weights each term by the inverse frequency of its appearance across different documents, and reduces the important of common words like "the".

In [ ]:
idf = {term: np.log((1 + N)/(1 + count)) for term, count in df.items()}

#### Term frequency - inverse document frequency

This is just the product of tf and idf, and a measure of the importance of each term in a document.

In [ ]:
terms = list(tz.unique(tz.concat(doc_terms)))

In [ ]:
tf_idf = {}
for name, doc in tf.items():
    d = {}
    for term in terms:
        d[term] = doc.get(term, 0) * idf[term]
    tf_idf[name] = d

#### Convert to a structured data type

In [ ]:
tf_idf = pd.DataFrame(tf_idf)

In [ ]:
tf_idf.iloc[:5, :5]

#### Distinctive words in each document

These are words that appear frequently in that document but not in others.

In [ ]:
pd.DataFrame({doc: series.sort_values(ascending=False).index[:5] 
              for doc, series in tf_idf.items()}).T

#### Why is "macb" the top hit for Macbeth? 

In [ ]:
tf['shakespeare-macbeth']['macb']

In [ ]:
text = open('data/gutenberg/shakespeare-macbeth.txt', encoding='latin-1').read()
re.findall('macb[^e].*\n', text, re.IGNORECASE)[:10]

### Image data

In [ ]:
paths = glob.glob(os.path.join('data', 'POKEMON', '*.png') )[:3]

imgs = np.array([imageio.imread(path) for path in paths])

In [ ]:
imgs.shape

In [ ]:
fig, axes = plt.subplots(1,3,figsize=(10,3))
for ax, img in zip(axes, imgs):
    ax.imshow(img)
    ax.set_xticks([])
    ax.set_yticks([])

## Using `pandas` for data munging

In [ ]:
url = "https://raw.githubusercontent.com/mwaskom/seaborn-data/master/tips.csv"

In [ ]:
tips = pd.read_csv(url)

### Inspecting a data frame

In [ ]:
tips.shape

In [ ]:
tips.dtypes

In [ ]:
tips.columns

In [ ]:
tips.index

In [ ]:
tips.describe()

In [ ]:
tips.head(3)

In [ ]:
tips.tail(3)

In [ ]:
tips.sample(3)

### Series

A column is a pandas Series object. It behaves like an indexed vector.

In [ ]:
tips['sex'].head()

Alternative way to get Series when column name meets requirements for Python variable (i.e. no spaces or punctuation)

In [ ]:
tips.sex.head()

### Series types

#### String operations

In [ ]:
tips.sex.str.lower().str[0].head()

#### Categorical data types

In [ ]:
tips.day.unique()

In [ ]:
tips['day'] = tips.day.astype('category')

In [ ]:
tips.day.head(3)

In [ ]:
tips.day.cat.reorder_categories(['Thur', 'Fri', 'Sat', 'Sun'], ordered=True, inplace=True)

In [ ]:
tips.day.head(3)

#### Datetime operations

In [ ]:
import pandas_datareader as pdr

data_source = 'google'
start_date = '2010-01-01'
end_date = '2016-12-31'

data = pdr.get_data_morningstar('MSFT', start_date, end_date)

In [ ]:
data.head(3)

In [ ]:
dates = data.index.get_level_values(1)

In [ ]:
dates[:3]

In [ ]:
list(it.islice(zip(dates.year, 
                    dates.month,
                    dates.day), 6))

In [ ]:
msft = data.loc['MSFT']

In [ ]:
msft.plot(y='High')
pass

In [ ]:
msft.loc['2016-12-01':'2016-12-31', ('Low', 'High')].plot(
    linestyle='dashed', marker='+')
pass

### Indexing

In [ ]:
tips[0:2]

In [ ]:
tips.loc[0:2]

In [ ]:
tips.iloc[0:2]

In [ ]:
tips.iloc[0:3, [2,3,4]]

In [ ]:
tips.loc[0:2, ['tip', 'sex', 'size']]

In [ ]:
tips.loc[0:2, 'tip':'day']

#### Boolean indexing

In [ ]:
tips[tips.sex == 'Male'].head(3)

In [ ]:
tips[(tips.sex == 'Male') & (tips.time != 'Dinner')].head(3)

### Special selection

In [ ]:
df = pd.DataFrame({
    'a': [1,2,None,4,1], 
    'b': [3,3,None,None,3],
    'c': [1,2,None,4,1]})

In [ ]:
df

In [ ]:
df.dropna()

In [ ]:
df.dropna(how='all')

In [ ]:
df.drop_duplicates()

In [ ]:
df.drop_duplicates(keep='last')

### Selecting by label

In [ ]:
tips.filter(regex='^s.*').head(3)

### Sorting

In [ ]:
tips.sort_values(['size', 'tip'], ascending=[True, False]).head(3)

Note that ordered categorical values are sorted appropriately.

In [ ]:
tips.sort_values(['day']).head(3)

### Rearrange columns

In [ ]:
tips = tips[tips.columns.sort_values()]
tips.head(3)

In [ ]:
tips = tips.filter('total_bill	tip	sex	smoker	day	time	size'.split())
tips.head(3)

### Transforms

In [ ]:
tips['day_type'] = 'Weekday'
tips.loc[tips.day.isin(['Sat', 'Sun']), 'day_type'] = 'Weekend'
tips.head(3)

In [ ]:
tips['cost'] = tips['total_bill'] + tips['tip']
tips.head(3)

In [ ]:
tips.assign(log1p_cost = np.log1p(tips.cost)).head(3)

In [ ]:
tips.replace({
    'sex': dict(Femals='F', Male='M'),
    'day': dict(Thur=4, Fri=5, Sat=6, Sun=7)}).head(3)

In [ ]:
tips.rename({'sex': 'gender'}, axis=1).head(3)

#### Transforming missing values

In [ ]:
df

In [ ]:
df.fillna(0)

In [ ]:
df.fillna(df.mean())

In [ ]:
df.fillna(method='ffill')

In [ ]:
df.fillna(method='bfill')

### Summaries

In [ ]:
tips.mean()

In [ ]:
tips.std()

In [ ]:
tips.count()

### Grouping

In [ ]:
tips.groupby(['sex', 'day', 'time']).mean()

In [ ]:
tips.groupby(['sex']).agg(['mean', 'std', 'min', 'max'])

### Working with hierarchical indexes

In [ ]:
df = tips.groupby(['sex', 'day', 'time']).mean()
df.head()

In [ ]:
df.index

In [ ]:
df.columns

In [ ]:
df.loc[('Female')]

In [ ]:
df.loc[('Female', 'Sat')]

In [ ]:
df.loc[('Female', slice(None), 'Dinner')]

### Stacking and unstacking

In [ ]:
tips.head(3)

In [ ]:
df.unstack()['tip']

In [ ]:
df.unstack(level=0)['tip']

In [ ]:
df.unstack(level=[1,2])['tip']

In [ ]:
df.unstack(level=[1,2])['tip'].stack(level=0)

#### Swapping levels

In [ ]:
df.swaplevel(1,2)

### Resetting index

If you'd rather not deal with hierarchical indexes, use `reset_index`.

In [ ]:
df.reset_index()

### Reshaping

In [ ]:
df1 = df.reset_index()

In [ ]:
df1.head(3)

#### Transpose

In [ ]:
df1.T

#### Melt (gather)

In [ ]:
pd.melt(df1, id_vars=['sex', 'day', 'time', 'size']).head(10)

### Pivot table

A pivot table is like a group_by operation on both the index (rows) and columns.

In [ ]:
pd.pivot_table(df1, values=['total_bill', 'tip'],
               index=['sex', 'day'], 
               columns='time',
               aggfunc='mean')

### Joining data frames

In [ ]:
tips1 = tips[0:5]
tips1

In [ ]:
tips2 = tips[3:8]
tips2

#### Simple concatenation

In [ ]:
pd.concat([tips1, tips2])

### Joining columns

Merge uses all common columns to combine. It is very flexible - see help(pd.merge).

In [ ]:
pd.merge(tips1, tips2)

In [ ]:
pd.merge(tips1, tips2, how='left')

In [ ]:
pd.merge(tips1, tips2, how='right')

In [ ]:
pd.merge(tips1, tips2, how='outer')

## Visualizing data

In [ ]:
tips.head()

In [ ]:
sns.set_context('notebook', font_scale=1.3)
g = sns.factorplot(x='sex', y='tip',
                   col='time', row='smoker',
                   data=tips, kind='swarm')
pass

In [ ]:
sns.set_context('notebook', font_scale=1.3)
g = sns.factorplot(x='day', y='tip', hue='sex',
                   col='time', row='smoker',
                   data=tips, kind='violin', 
                   margin_titles=True)
pass

In [ ]:
sns.lmplot(x='total_bill', y='tip', hue='sex', 
           col='time', row='smoker', 
           data=tips, palette='dark')
pass

### Pipelines

As nearly all `pandas` functions and methods consume and return a `DataFrame`, they can be chained togtehr to construct a pipeline. 

In [ ]:
tips.head()

In [ ]:
pd.set_option('precision', 2)

In [ ]:
(
    tips.
    dropna().
    filter(regex=r".*[t|s].*").
    loc[lambda df: df.sex=='Female'].
    groupby(['smoker', 'time']).
    agg({'total_bill': 'mean', 
         'tip': 'mean'}).
    assign(percent = lambda df: 100*df.tip/df.total_bill)
)